In [1]:
import torch
import os
import csv
from tqdm import tqdm
from helpers_function import *
from data import get_data
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
# from langchain.embeddings import HuggingFaceEmbeddings

# tokenizer = AutoTokenizer.from_pretrained("valhalla/longformer-base-4096-finetuned-squadv1")
# Model = AutoModelForQuestionAnswering.from_pretrained("valhalla/longformer-base-4096-finetuned-squadv1", return_dict=False)

c:\ProgramData\Anaconda3\envs\pytorch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
urls, company_names, segment = get_data()

In [3]:
model = 'valhalla'
merge_data(model, company_names)

In [4]:
# questions = ['what is the name of this company?', 'when was this company established?', 'what is the company ISO certification?', 'what is the total number of employees?', 'what is the growth rate of this company?', 'what is the revenue of this company?', 'where is the global HQ location of this company?', 'on what field this company works?', 'what is the vision of this company?', 'what are the past awards that has been awarded to this company?', 'Describe about this company.', 'what is the name of the parent company?', 'what is the company external ICT spend?', 'what is the company R&D spend?', 'what is the company R&D focus?', 'what is the marketing budget of this company?', 'what is the number of patents of this company?']
questions = ['what is the name of the parent company?', 'when was this company established?', 'Describe about this company.', 'where is the global Headquater of this company?', 'where is the geographical presence of this company?', 'who is the owner of this company?', 'what are the core business units of this company?', 'what is the total number of subsidiaries?', 'what is the total number of employees?', 'what is the total revenue of this company?', 'what is the growth rate of this company?', 'what is the operating profits of this company?', 'Revenue by Region of this company?', 'what are the venture investments?', 'External ICT spend?', 'total marketing budget?', 'what is the R&D focus of this company?', 'what is the R&D spend of this company?', 'total number of patents of this company?', 'what is the vision of this company?', 'what are the strategic priorities?', 'who are the partners of this company?', 'what is the ISO certification of this company?', 'what are the past awards that has been awarded to this company?']
print(len(questions))

24


In [5]:
# ans_path = "D:/Machine Learning/scrapper and extraction/valhalla_answers"

# if not os.path.exists(ans_path):
#     os.mkdir(ans_path)

# final_answer = []

# for i, company in tqdm(enumerate(company_names)):
#     answers = []
#     try:
#         for j in range(5):
#             path = f"D:/Machine Learning/scrapper and extraction/raw_text/{company}/my_file_{j}.txt"
#             text = get_text(path)

#             if text == "":
#                 continue

#             ans = []

#             for ques in questions:
#                 text = text
#                 question = ques
#                 encoding = tokenizer(question, text, return_tensors="pt")
#                 input_ids = encoding["input_ids"]

#                 # default is local attention everywhere
#                 # the forward method will automatically set global attention on question tokens
#                 attention_mask = encoding["attention_mask"]

#                 start_scores, end_scores = Model(input_ids, attention_mask=attention_mask)
#                 all_tokens = tokenizer.convert_ids_to_tokens(input_ids[0].tolist())

#                 answer_tokens = all_tokens[torch.argmax(start_scores) :torch.argmax(end_scores)+1]
#                 answer = tokenizer.decode(tokenizer.convert_tokens_to_ids(answer_tokens))

#                 ans.append(answer)
            
#             answers.append(ans)
    
#     except:
#         continue

#     final_answer.append(answers)

In [6]:
# ans_path = "D:/Machine Learning/scrapper and extraction/valhalla_answers"
final_ans = []
# embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2") # Equivalent to SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
for i, company in tqdm(enumerate(company_names)):
    tokenizer = AutoTokenizer.from_pretrained("valhalla/longformer-base-4096-finetuned-squadv1")
    Model = AutoModelForQuestionAnswering.from_pretrained("valhalla/longformer-base-4096-finetuned-squadv1", return_dict=False)
    embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

    directory = f"D:/Machine Learning/scrapper and extraction/Valhalla/data/{company}"
    docs = load_documents(directory)
    docs = split_documents(docs)
    ans = []

    for ques in questions:
        ques = ques
        text = get_relevant_docs(docs, ques, embeddings)
        encoding = tokenizer(ques, text, return_tensors="pt")
        input_ids = encoding["input_ids"]
        # default is local attention everywhere
        # the forward method will automatically set global attention on question tokens
        attention_mask = encoding["attention_mask"]

        start_scores, end_scores = Model(input_ids, attention_mask=attention_mask)
        all_tokens = tokenizer.convert_ids_to_tokens(input_ids[0].tolist())

        answer_tokens = all_tokens[torch.argmax(start_scores) :torch.argmax(end_scores)+1]
        answer = tokenizer.decode(tokenizer.convert_tokens_to_ids(answer_tokens))

        ans.append(answer)
    
    final_ans.append(ans)

    

0it [00:00, ?it/s]Some weights of the model checkpoint at valhalla/longformer-base-4096-finetuned-squadv1 were not used when initializing LongformerForQuestionAnswering: ['longformer.pooler.dense.weight', 'longformer.pooler.dense.bias']
- This IS expected if you are initializing LongformerForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
1it [06:06, 366.51s/it]Some weights of the model checkpoint at valhalla/longformer-base-4096-finetuned-squadv1 were not used when initializing LongformerForQuestionAnswering: ['longformer.pooler.dense.weight', 'longformer.pooler.dense.bias']
- This IS expected if you 

In [16]:
print(final_ans[1])
print(len(final_ans[4]))
print(final_ans[1]==final_ans[3])

['Woebot Health', ' 7th annual', 'LG NOVA Mission for the Future ProgramThe LG Mission for the Future is a global engagement program for startups and entrepreneurs sponsored by LG NOVA – LG’s new innovation center', '', ' outside', ' LG NOVA', ' startups and entrepreneurs', ' third party beneficiary', ' 100', '', '<s>what is the growth rate of this company?</s></s>This is an opportunity to streamline innovation efforts into a sustainable and proactive process that incorporates leading employer insights to identify and prioritize emerging solutions for partnerships.MedTech Breakthrough AwardWoebot Health', '', ' Woebot', '<s>', ' free', '<s>total marketing budget?</s></s>This meeting serves as a chance for you to connect with the visionary leaders of our organization and gain a deeper understanding of our company’s values, long-term goals, and culture. It’s also an opportunity for the founders to get to know you better and assess how you align with our company’s vision. This will be in 

In [8]:
# for i, company in enumerate(company_names):
#     file = f"D:/Machine Learning/scrapper and extraction/valhalla_answers/{company}.txt"
#     with open(file, "w") as f:
#         for k, ques in enumerate(questions):
#             f.write(f'Questions: {ques}\nAnswer: {[ans[k] for ans in final_answer[i]]}\n')

In [18]:
path = r'D:\Machine Learning\scrapper and extraction\Valhalla\answers'

os.mkdir(path)

for i, company in enumerate(company_names):
    file = f"{path}/{company}.csv"
    with open(file, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["Questions", "Answers"])
        for k, ques in enumerate(questions):
            writer.writerow([ques, final_ans[i][k]])
